<a href="https://colab.research.google.com/github/unnatiuppal/promptengineering/blob/main/File3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


import json

# --- 1. Simulation of the LLM Call ---
def call_llm(prompt: str, input_data: dict = None) -> str:
    """
    A placeholder function to simulate a call to a Large Language Model (LLM).
    In a real application, this would be an API call (e.g., to Gemini, OpenAI, etc.).
    It uses the prompt and input_data to produce a simulated output.
    """
    print(f"\n--- Running LLM Step: {prompt.splitlines()[0][:30]}... ---")

    # Simple, deterministic simulation based on the prompt's content
    if "Triage AI" in prompt:
        # Simulate P-1: Triage and Extraction
        user_message = input_data.get("UserMessage", "")
        if "12345" in user_message:
            output_data = {
                "Intent": "Delivery Issue",
                "OrderID": "12345",
                "OriginalQuery": user_message
            }
        elif "67890" in user_message:
            output_data = {
                "Intent": "Delivery Issue",
                "OrderID": "67890",
                "OriginalQuery": user_message
            }
        else:
            output_data = {
                "Intent": "General Inquiry",
                "OrderID": "N/A",
                "OriginalQuery": user_message
            }
        return json.dumps(output_data)

    elif "Support Diagnostic AI" in prompt:
        # Simulate P-2: Diagnosis and Response
        intent = input_data.get("Intent")
        order_id = input_data.get("OrderID")

        if intent == "Delivery Issue" and order_id == "12345":
            response = "Thank you for providing your Order ID 12345. I see that your laptop is currently delayed in transit, but it's now estimated to arrive by the end of tomorrow. I apologize for the inconvenience."
        elif intent == "Delivery Issue" and order_id == "67890":
            response = "I see order 67890 is still pending shipment. Please check back in 24 hours for tracking details."
        else:
            response = "Thank you for reaching out. Let me look into your general inquiry right away."

        # P-2's output is the customer-facing response, and we wrap it for the next step
        return json.dumps({"AgentResponse": response, "Intent": intent, "OrderID": order_id})

    elif "Handover AI" in prompt:
        # Simulate P-3: Escalation/Next Step
        response = input_data.get("AgentResponse", "")
        original_query = input_data.get("OriginalQuery", "") # Re-injecting original query for escalation note

        if "estimated to arrive by the end of tomorrow" in response:
            return "STATUS: RESOLVED"
        else:
            # If the response was not fully resolving, escalate
            return f"STATUS: ESCALATE\nInternal Note: Customer reported issue: '{original_query}'. Diagnostic response: '{response}'. Requires human agent follow-up."

    return "Simulated Error: Unknown Prompt Type"


# --- 2. Define Prompts (as per the structure) ---

# P-1: Initial Triage and Intent Classification
PROMPT_1_TRIAGE = """
You are a Customer Support Triage AI. Your goal is to identify the user's main issue and extract relevant data. Analyze the customer message below and classify the intent into one of the following: `Delivery Issue`, `Billing Query`, `Product Defect`, `General Inquiry`. Then, extract the relevant Order ID, if provided.
Return the output strictly as a JSON object.

Customer Message: {user_message}
"""

# P-2: Information Retrieval and Initial Diagnosis
PROMPT_2_DIAGNOSIS = """
You are a Support Diagnostic AI. You have classified the issue as {intent} with Order ID {order_id}. Search your knowledge base to provide a precise, empathetic, and actionable first response.
The simulated Knowledge Base Status is:
- If Delivery Issue & OrderID is 12345: Status is "Delayed in Transit: Estimated delivery now tomorrow."
- If Delivery Issue & OrderID is 67890: Status is "Pending Shipment: Please check back in 24 hours."

Generate a customer-facing response, then wrap it in a JSON object with the key "AgentResponse".
"""

# P-3: Escalation/Next Step Generation
PROMPT_3_HANDOVER = """
You are a Handover AI. Based on the initial diagnosis, determine the necessary follow-up.
If the Diagnosis Response is fully resolving (e.g., provides a clear ETA or immediate answer), output "STATUS: RESOLVED".
If it requires a human review (e.g., needs action beyond simple lookup), output "STATUS: ESCALATE" and draft an internal note for the human agent.

Diagnosis Response: {diagnosis_response}
"""


# --- 3. The Prompt Chain Execution Function ---

def run_support_chain(user_input: str):
    """Executes the three-step prompt chain."""

    print(f"\n{'='*50}\n👤 CUSTOMER INPUT: {user_input}\n{'='*50}")

    # ----------------------------------------
    # STEP 1: P-1 Triage and Extraction
    # ----------------------------------------
    p1_input = {"UserMessage": user_input}
    p1_prompt = PROMPT_1_TRIAGE.format(user_message=user_input)
    p1_output_raw = call_llm(p1_prompt, p1_input)

    try:
        p1_output = json.loads(p1_output_raw)
        chain_context = p1_output # Initialize context for subsequent steps
        print(f"✅ STEP 1 Output (Triage): {chain_context}")
    except json.JSONDecodeError:
        print("❌ ERROR: P-1 failed to return valid JSON.")
        return

    # ----------------------------------------
    # STEP 2: P-2 Diagnosis and Response
    # ----------------------------------------
    p2_prompt = PROMPT_2_DIAGNOSIS.format(
        intent=chain_context['Intent'],
        order_id=chain_context['OrderID']
    )
    p2_output_raw = call_llm(p2_prompt, chain_context)

    try:
        p2_output = json.loads(p2_output_raw)
        agent_response = p2_output['AgentResponse']
        chain_context['AgentResponse'] = agent_response # Update context
        print(f"✅ STEP 2 Output (Agent Response): {agent_response}")
    except json.JSONDecodeError:
        print("❌ ERROR: P-2 failed to return valid JSON.")
        return

    # ----------------------------------------
    # STEP 3: P-3 Escalation/Handover
    # ----------------------------------------
    p3_prompt = PROMPT_3_HANDOVER.format(diagnosis_response=agent_response)
    p3_input = chain_context # Pass full context including original query for escalation
    p3_final_status = call_llm(p3_prompt, p3_input)

    print(f"\n{'*'*50}")
    print(f"🤖 FINAL ACTION / HANDOVER STATUS:\n{p3_final_status}")
    print(f"{'*'*50}\n")


# --- 4. Execution Examples ---

# Example 1: Resolvable Delivery Issue (Order 12345)
user_message_1 = "I ordered a new laptop last week, order #12345, and the tracking hasn't updated in three days. I need to know where it is!"
run_support_chain(user_message_1)

# Example 2: Non-Resolvable Delivery Issue (Order 67890 - needs more time)
user_message_2 = "My order 67890 still says 'pending shipment'. What is going on?"
run_support_chain(user_message_2)

# Example 3: General Inquiry (Non-resolvable without database access)
user_message_3 = "What is your return policy for open-box items?"
run_support_chain(user_message_3)




👤 CUSTOMER INPUT: I ordered a new laptop last week, order #12345, and the tracking hasn't updated in three days. I need to know where it is!

--- Running LLM Step: ... ---
✅ STEP 1 Output (Triage): {'Intent': 'Delivery Issue', 'OrderID': '12345', 'OriginalQuery': "I ordered a new laptop last week, order #12345, and the tracking hasn't updated in three days. I need to know where it is!"}

--- Running LLM Step: ... ---
✅ STEP 2 Output (Agent Response): Thank you for providing your Order ID 12345. I see that your laptop is currently delayed in transit, but it's now estimated to arrive by the end of tomorrow. I apologize for the inconvenience.

--- Running LLM Step: ... ---

**************************************************
🤖 FINAL ACTION / HANDOVER STATUS:
STATUS: RESOLVED
**************************************************


👤 CUSTOMER INPUT: My order 67890 still says 'pending shipment'. What is going on?

--- Running LLM Step: ... ---
✅ STEP 1 Output (Triage): {'Intent': 'Delivery Issu